Processing full covariance files - generate invcov, SNR, modified cov and its inverse for each mask

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from numpy import linalg as LA
import sys
import cosmolike_libs_LSSTxSO_10x2pt as clike
import os
import importlib
sys.path.append('cov_proc')
import cov_proc as cp

In [2]:
LSST, ROMAN, ROMAN_WIDE, ROMAN_WIDE2=0, 0, 0, 1
lsst_s3000 = 0
CMB_SO, CMB_S4 = 0, 1
ONESAMPLE=1

In [3]:
if (ROMAN):
    if (CMB_SO):
        raw_cov_file = 'cov/cov_romanxso'
        full_fid_datav = 'datav/10x2pt_RomanxSO'
    else:
        raw_cov_file = 'cov/cov_romanxs4'
        full_fid_datav = 'datav/10x2pt_RomanxS4'
if (ROMAN_WIDE):
    if (CMB_SO):
        raw_cov_file = 'cov/cov_romanwidexso'
        full_fid_datav = 'datav/10x2pt_RomanWidexSO'
    else:
        raw_cov_file = 'cov/cov_romanwidexs4'
        full_fid_datav = 'datav/10x2pt_RomanWidexS4'
if (ROMAN_WIDE2):
    if (CMB_SO):
        raw_cov_file = 'cov/cov_romanwide2xso'
        full_fid_datav = 'datav/10x2pt_RomanWide2xSO'
    else:
        raw_cov_file = 'cov/cov_romanwide2xs4'
        full_fid_datav = 'datav/10x2pt_RomanWide2xS4'
if (LSST):
    raw_cov_file = 'cov/cov_lsstxso_y6'
    if (lsst_s3000):
        full_fid_datav = 'datav/10x2pt_LSSTxSO_Y6_s3000'
    else:
        full_fid_datav = 'datav/10x2pt_LSSTxSO_Y6'

if (ONESAMPLE):
    raw_cov_file = raw_cov_file + '_1sample'
    full_fid_datav = full_fid_datav + '_1sample'
    
full_mask = full_fid_datav + '_mask.txt'

In [4]:
!python datav/mask_c_dv.py {full_fid_datav}

mask file datav/10x2pt_RomanWide2xS4_1sample_mask.txt saved!


Get compressed covariance and test

In [5]:
raw_cov = cp.RawCov(raw_cov_file)

reading cov: cov/cov_romanwide2xs4_1sample
row counts: 7363125
size: 3825 x 3825
Compressed covariance file saved at cov/cov_romanwide2xs4_1sample_comp


In [6]:
fullcov = cp.Cov(matrix = raw_cov.covmat)
# fullcov = cp.Cov(path = 'cov/cov_lsstxso_y6_comp')

# check full cov
fullcov.test_eigens()

covmat assigned, dimension: 3825 x 3825
sorted eigenvalues:  [5.92493402e-44 1.44147890e-43 6.28601777e-43 ... 2.98781470e-11
 4.45513894e-11 6.97130331e-11]
Eigenvalues look good!


In [7]:
mask = cp.Mask(full_mask)

mask set.


In [8]:
N_10x2pt = mask.mask.size
N_len = N_src = 10
N_ell = 25
N_ss = N_src * (N_src + 1) // 2
N_shear = N_ss * N_ell
N_6x2pt = N_10x2pt - (N_len + N_src + 2) * N_ell
N_3x2pt = N_6x2pt - (N_len + N_src + 1) * N_ell
N_8x2pt = N_10x2pt - (N_len + 1) * N_ell
print(f'N_shear={N_shear}, N_3x2pt={N_3x2pt}, N_6x2pt={N_6x2pt}, N_10x2pt={N_10x2pt}, N_8x2pt={N_8x2pt}')

mask_nx2pt = [cp.Mask() for _ in range(4)] # 3/6/10/8x2pt
mask_nx2pt[2] = mask
mask_nx2pt[0].mask = mask.mask[:N_3x2pt]
mask_nx2pt[1].mask = mask.mask[:N_6x2pt]
mask_nx2pt[3].mask = np.concatenate( (mask.mask[:N_6x2pt], mask.mask[N_6x2pt+N_len*N_ell: N_10x2pt-2*N_ell], mask.mask[-N_ell:]) )

mask_name = ['3x2pt', '6x2pt', '10x2pt', '8x2pt']
datav_fid = np.loadtxt(full_fid_datav)[:,1]
datav_fid_8x2pt = np.concatenate( (datav_fid[:N_6x2pt], datav_fid[N_6x2pt+N_len*N_ell: N_10x2pt-2*N_ell], datav_fid[-N_ell:]) )
datavs_fid = [datav_fid[:N_3x2pt], datav_fid[:N_6x2pt], datav_fid, datav_fid_8x2pt]
for i in [0,1,3]:
    cur_mask = mask_nx2pt[i].mask
    np.savetxt(full_mask.replace("10x2pt_", mask_name[i]+"_"), np.c_[np.arange(cur_mask.size), cur_mask], fmt="%d %d")
    np.savetxt(full_fid_datav.replace("10x2pt_", mask_name[i]+"_"), np.c_[np.arange(cur_mask.size), datavs_fid[i]], fmt="%d %le")
    print(f'mask {mask_name[i]} saved! datav saved!')

N_shear=1375, N_3x2pt=2750, N_6x2pt=3275, N_10x2pt=3825, N_8x2pt=3550
Mask object must be initialized with file path
Mask object must be initialized with file path
Mask object must be initialized with file path
Mask object must be initialized with file path
mask 3x2pt saved! datav saved!
mask 6x2pt saved! datav saved!
mask 8x2pt saved! datav saved!


In [9]:
for i in range(3):
    snr = fullcov.get_snr(full_fid_datav.replace("10x2pt_", mask_name[i]+"_"), full_mask, inds=np.arange(mask_nx2pt[i].mask.size))
    print("%s:"%mask_name[i], snr)

i=3 # 8x2pt
inds_8x2pt = np.concatenate( (np.arange(N_6x2pt), np.arange(N_6x2pt+N_len*N_ell, N_10x2pt-2*N_ell), np.arange(N_10x2pt-N_ell, N_10x2pt)) )
snr = fullcov.get_snr(full_fid_datav.replace("10x2pt_", mask_name[i]+"_"), full_mask, inds=inds_8x2pt)
print("%s:"%mask_name[i], snr)

mask_yy = cp.Mask('datav/mask_yy.txt')
dvfile_yy = "datav/yy_fid"
snr = fullcov.get_snr(dvfile_yy, mask=mask_yy, inds=np.arange(N_10x2pt-N_ell, N_10x2pt) )
print("yy:", snr)

mask_kk = cp.Mask('datav/mask_kk.txt')
dvfile_kk = "datav/kk_fid"
snr = fullcov.get_snr(dvfile_kk, mask=mask_kk, inds=np.arange(N_6x2pt-N_ell, N_6x2pt) )
print("kk:", snr)

3x2pt: 1320.0576358951955
6x2pt: 1364.2243027083493
10x2pt: 1531.4962919358945
8x2pt: 1452.720318366397
mask set.
yy: 262.0179376019224
mask set.
kk: 262.5909791971263


In [10]:
# inv='invcov_Y6_10x2pt_modified'

# data='10x2pt_fid'

# mask='mask_10x2pt.txt'

# LSST Y6xSO
if (LSST):
    source_z='src_LSSTY6'
    lens_z='lens_LSSTY6'
    sigma_z_shear=0.05
    sigma_z_clustering=0.03
    survey_designation="LSSTxSO_Y6"
    tomo_binning_source="source_std"
    tomo_binning_lens="LSST_gold"
    if (lsst_s3000):
        lmax_shear=3000.0
        survey_designation = survey_designation + "_s3000"
    else:
        lmax_shear=7979.0
    # not used:
    nsource_table=22.5  
    nlens_table=26.8
    area_table=16500.0
    #

which_cmb = "so_Y5" # default
# RomanxSO / S4
if (ROMAN):
    source_z='zdistri_WFIRST_LSST_lensing_fine_bin_norm'
    lens_z='zdistri_WFIRST_LSST_clustering_fine_bin_norm'
    sigma_z_shear=0.01
    sigma_z_clustering=0.01
    if (CMB_SO):
        survey_designation="RomanxSO"
    else:
        survey_designation="RomanxS4"
        which_cmb = "s4"
    tomo_binning_source="source_std"
    tomo_binning_lens="WF_SN10"
    lmax_shear=4000.0
    # not used:
    nsource_table=51.0  
    nlens_table=66.0
    area_table=2000.0
    #
# RomanWidexSO / S4
if (ROMAN_WIDE or ROMAN_WIDE2):
    which_roman = "RomanWide"
    if ROMAN_WIDE2: which_roman = "RomanWide2"

    source_z='zdistri_WFIRST_LSST_lensing_fine_bin_norm'
    lens_z='zdistri_WFIRST_LSST_clustering_fine_bin_norm'
    sigma_z_shear=0.05
    sigma_z_clustering=0.02
    if (CMB_SO):
        survey_designation=f"{which_roman}xSO"
    else:
        survey_designation=f"{which_roman}xS4"
        which_cmb = "s4"
    tomo_binning_source="source_std"
    tomo_binning_lens="WF_SN10"
    lmax_shear=4000.0
    # not used:
    nsource_table=43.0
    nlens_table=50.0
    area_table=18000.0
    #

if (ONESAMPLE):
    lens_z = source_z
    sigma_z_clustering = sigma_z_shear
    survey_designation = survey_designation + '_1sample'
    tomo_binning_lens = "lens=src"
    nlens_table = nsource_table

## priors, not used for datav
shear_prior=0.003
delta_z_prior_shear=0.001
delta_z_prior_clustering=0.001 ## Not SRD, assumed to be no worse than shear
sigma_z_prior_shear=0.003
sigma_z_prior_clustering=0.003 ## Not SRD, assumed to be no worse than shear
##


file_source_z = os.path.join(clike.dirname, "zdistris/",source_z)
file_lens_z = os.path.join(clike.dirname, "zdistris/",lens_z)
# data_file = os.path.join(clike.dirname, "datav/",data)
# cov_file = os.path.join(clike.dirname, "cov/",inv)
# mask_file = os.path.join(clike.dirname, "datav/",mask)

clike.initcosmo("halomodel".encode('utf-8'))
# clike.initbins(25,20.0,7979.0,3000.0,21.0,10,10)
clike.initbins(25,20.0,7979.0,lmax_shear,21.0,10,10)

clike.initpriors(shear_prior,sigma_z_shear,delta_z_prior_shear,sigma_z_prior_shear,sigma_z_clustering,delta_z_prior_clustering,sigma_z_prior_clustering)
clike.initsurvey(survey_designation.encode('utf-8'),nsource_table,nlens_table,area_table)
clike.initgalaxies(file_source_z.encode('utf-8'),file_lens_z.encode('utf-8'),"gaussian".encode('utf-8'),"gaussian".encode('utf-8'),tomo_binning_source.encode('utf-8'),tomo_binning_lens.encode('utf-8'))
clike.initia("NLA_z".encode('utf-8'),"none".encode('utf-8'))

# test also with
#initpriors("none","none","none","Planck")
#initpriors("none","none","none","random")
clike.initprobes("10x2pt".encode('utf-8'))
# clike.initdatainv(cov_file.encode('utf-8'),data_file.encode('utf-8'),mask_file.encode('utf-8'))
clike.initcmb(which_cmb.encode('utf-8'))
clike.initfb(1)


0

In [11]:
omega_m=0.3156
sigma_8=0.831
n_s=0.9645
w0=-1.
wa=0.0
omega_b=0.0491685
h0=0.6727
MGSigma=MGmu=0
icp = clike.InputCosmologyParams(omega_m,sigma_8,n_s,w0,wa,omega_b,h0,MGSigma,MGmu)

inp = clike.InputNuisanceParams()
# inp.bias[:]=[1.2,1.3,1.38,1.46,1.54,1.63,1.72,1.83,1.94,2.08]

# LSST gold sample 
if (LSST):
    inp.bias[:]=[1.202462,1.313657,1.406308,1.493877,1.580479,1.668860,1.761558,1.860530,1.968723,2.090681]
    inp.source_z_s=0.05
    inp.lens_z_s=0.03
    src_z_s_fid, lens_z_s_fid = 0.05, 0.03
    if (ONESAMPLE):
        inp.lens_z_s=inp.source_z_s
        lens_z_s_fid = src_z_s_fid
        inp.bias[:]=[1.125013,1.308513,1.433975,1.558521,1.693127,1.844866,2.026988,2.265804,2.633997,4.047753]


if (ROMAN or ROMAN_WIDE or ROMAN_WIDE2): # Roman lens sample, gbias = 1.3 + 0.1*i, bin index i=0~9
    inp.bias[:]=[1.3 + 0.1*i for i in range(10)]
    if ROMAN:
        inp.source_z_s=0.01
        inp.lens_z_s=0.01
        src_z_s_fid, lens_z_s_fid = 0.01, 0.01
    else:
        inp.source_z_s=0.05
        inp.lens_z_s=0.02
        src_z_s_fid, lens_z_s_fid = 0.05, 0.02
        
    if (ONESAMPLE): # gbias use gold sample formula
        inp.lens_z_s=inp.source_z_s
        lens_z_s_fid = src_z_s_fid
        inp.bias[:]=[1.166664,1.403981,1.573795,1.744325,1.925937,2.131001,2.370211,2.651007,3.036247,4.556622]


inp.source_z_bias[:]=[0 for _ in range(10)]
inp.lens_z_bias[:]=[0 for _ in range(10)]
inp.shear_m[:]=[0 for _ in range(10)]
inp.A_ia=0.5
inp.eta_ia=0.0
inp.gas[:]=[1.17, 0.6, 14., 1., 0.03, 12.5, 1.2, 6.5, 0.752, 0., 0.]


In [12]:
dv_detail = survey_designation+"_fid"
clike.compute_data_vector(dv_detail.encode('utf-8'), icp, inp)

2026176612

In [13]:
for i in range(10):
    inp.shear_m[i] = 0.001
    dv_detail = survey_designation+"_m%d"%(i)
    clike.compute_data_vector(dv_detail.encode('utf-8'), icp, inp)
    print("datav m%d finished"%(i))
    inp.shear_m[:]=[0 for _ in range(10)]

if (ONESAMPLE):
    for i in range(10):
        inp.source_z_bias[i] = inp.lens_z_bias[i] = 0.001
        dv_detail = survey_designation+"_zls%d"%(i)
        clike.compute_data_vector(dv_detail.encode('utf-8'), icp, inp)
        print("datav zls%d finished"%(i))
        inp.source_z_bias[:]=[0 for _ in range(10)]
        inp.lens_z_bias[:]=[0 for _ in range(10)]
    
    inp.source_z_l = inp.source_z_s = src_z_s_fid+0.001
    dv_detail = survey_designation+"_zls-sig"
    clike.compute_data_vector(dv_detail.encode('utf-8'), icp, inp)
    print("datav zls-sig finished")
    inp.source_z_l = inp.source_z_s = src_z_s_fid

else: # Normal
    for i in range(10):
        inp.source_z_bias[i] = 0.001
        dv_detail = survey_designation+"_zs%d"%(i)
        clike.compute_data_vector(dv_detail.encode('utf-8'), icp, inp)
        print("datav zs%d finished"%(i))
        inp.source_z_bias[:]=[0 for _ in range(10)]

    for i in range(10):
        inp.lens_z_bias[i] = 0.001
        dv_detail = survey_designation+"_zl%d"%(i)
        clike.compute_data_vector(dv_detail.encode('utf-8'), icp, inp)
        print("datav zl%d finished"%(i))
        inp.lens_z_bias[:]=[0 for _ in range(10)]

    inp.source_z_s = src_z_s_fid+0.001
    dv_detail = survey_designation+"_zs-sig"
    clike.compute_data_vector(dv_detail.encode('utf-8'), icp, inp)
    print("datav zs-sig finished")
    inp.source_z_s = src_z_s_fid

    inp.lens_z_s = lens_z_s_fid+0.001
    dv_detail = survey_designation+"_zl-sig"
    clike.compute_data_vector(dv_detail.encode('utf-8'), icp, inp)
    print("datav zl-sig finished")
    inp.lens_z_s = lens_z_s_fid

datav m0 finished
datav m1 finished
datav m2 finished
datav m3 finished
datav m4 finished
datav m5 finished
datav m6 finished
datav m7 finished
datav m8 finished
datav m9 finished
datav zls0 finished
datav zls1 finished
datav zls2 finished
datav zls3 finished
datav zls4 finished
datav zls5 finished
datav zls6 finished
datav zls7 finished
datav zls8 finished
datav zls9 finished
datav zls-sig finished


In [14]:
if(LSST):
    prior_ms_sig = 0.003
    prior_dzlens_sig = prior_dzsrc_sig = 0.001
    prior_sigzlens_sig = prior_sigzsrc_sig = 0.003
if(ROMAN or ROMAN_WIDE or ROMAN_WIDE2):
    prior_dzlens_sig = prior_dzsrc_sig = 0.001
    prior_ms_sig = prior_sigzlens_sig = prior_sigzsrc_sig = 0.002

In [15]:
dv = datav_fid

In [16]:
cov_ms = np.zeros(fullcov.covmat.shape)
sigsqr_ms = prior_ms_sig**2
for i in range(10):
    dv_ms = (np.loadtxt(f'datav/10x2pt_{survey_designation}_m{i}'))[:,1]
    dv_ms_deriv = (dv_ms - dv)/0.001
    cov_ms += sigsqr_ms * np.dot(dv_ms_deriv.reshape((dv.size,1)),dv_ms_deriv.reshape(1,dv.size))
print('obtain cov_ms')

obtain cov_ms


In [17]:
if (ONESAMPLE):
    cov_phz = np.zeros(fullcov.covmat.shape)
    sigsqr_dz = prior_dzsrc_sig**2
    for i in range(10):
        dv_zls = (np.loadtxt(f'datav/10x2pt_{survey_designation}_zls{i}'))[:,1]
        dv_zls_deriv = (dv_zls - dv)/0.001
        cov_phz += sigsqr_dz * np.dot(dv_zls_deriv.reshape((dv.size,1)),dv_zls_deriv.reshape(1,dv.size))

    sigsqr_zsig = prior_sigzsrc_sig**2
    dv_zls = (np.loadtxt(f'datav/10x2pt_{survey_designation}_zls-sig'))[:,1]
    dv_zls_deriv = (dv_zls - dv)/0.001
    cov_phz += sigsqr_zsig * np.dot(dv_zls_deriv.reshape((dv.size,1)),dv_zls_deriv.reshape(1,dv.size))

    print('obtain cov_phz')

else: # Normal
    cov_phz_src = np.zeros(fullcov.covmat.shape)
    sigsqr_dz = prior_dzsrc_sig**2
    for i in range(10):
        dv_zs = (np.loadtxt(f'datav/10x2pt_{survey_designation}_zs{i}'))[:,1]
        dv_zs_deriv = (dv_zs - dv)/0.001
        cov_phz_src += sigsqr_dz * np.dot(dv_zs_deriv.reshape((dv.size,1)),dv_zs_deriv.reshape(1,dv.size))

    sigsqr_zsig = prior_sigzsrc_sig**2
    dv_zs = (np.loadtxt(f'datav/10x2pt_{survey_designation}_zs-sig'))[:,1]
    dv_zs_deriv = (dv_zs - dv)/0.001
    cov_phz_src += sigsqr_zsig * np.dot(dv_zs_deriv.reshape((dv.size,1)),dv_zs_deriv.reshape(1,dv.size))

    print('obtain cov_phz_src')

    cov_phz_lens = np.zeros(fullcov.covmat.shape)
    sigsqr_dz = prior_dzlens_sig**2
    for i in range(10):
        dv_zl = (np.loadtxt(f'datav/10x2pt_{survey_designation}_zl{i}'))[:,1]
        dv_zl_deriv = (dv_zl - dv)/0.001
        cov_phz_lens += sigsqr_dz * np.dot(dv_zl_deriv.reshape((dv.size,1)),dv_zl_deriv.reshape(1,dv.size))

    dv_zl = (np.loadtxt(f'datav/10x2pt_{survey_designation}_zl-sig'))[:,1]
    dv_zl_deriv = (dv_zl - dv)/0.001
    cov_phz_lens += sigsqr_zsig * np.dot(dv_zl_deriv.reshape((dv.size,1)),dv_zl_deriv.reshape(1,dv.size))

    print('obtain cov_phz_lens')
    cov_phz = cov_phz_lens + cov_phz_src

obtain cov_phz


In [18]:
# frac_cov_phz_lens = cov_phz_lens/(fullcov.covmat+cov_phz_lens)
# frac_cov_phz_src = cov_phz_src/(fullcov.covmat+cov_phz_src)
# plt.figure(figsize=(15,15))
# im = plt.imshow(frac_cov_phz_lens, cmap='seismic', vmin=-1, vmax=1)
# plt.colorbar(im, orientation='vertical')
# plt.show()
# plt.figure(figsize=(15,15))
# im = plt.imshow(frac_cov_phz_src, cmap='seismic', vmin=-1, vmax=1)
# plt.colorbar(im, orientation='vertical')
# plt.show()

In [19]:
cov_modified = cp.Cov(matrix = fullcov.covmat + cov_ms + cov_phz)
cp.cov_3col_out(cov_modified.covmat, raw_cov_file+'_modified')

covmat assigned, dimension: 3825 x 3825
cov saved to cov/cov_romanwide2xs4_1sample_modified!


In [20]:
# importlib.reload(cp)

In [21]:
Ndata = [N_3x2pt, N_6x2pt, N_10x2pt, N_8x2pt]
outname = '_modified_'
if (LSST and lsst_s3000):
    outname = outname + 's3000_'

for i in range(3):
    invcov_modified = cov_modified.get_invcov_masked(mask=mask.mask, inds=np.arange(Ndata[i]))
    cp.cov_3col_out(invcov_modified, raw_cov_file.replace('cov_', 'invcov_') + outname + mask_name[i])

i=3
invcov_modified = cov_modified.get_invcov_masked(mask=mask.mask, inds=inds_8x2pt)
cp.cov_3col_out(invcov_modified, raw_cov_file.replace('cov_', 'invcov_') + outname + mask_name[i])


cov saved to cov/invcov_romanwide2xs4_1sample_modified_3x2pt!
cov saved to cov/invcov_romanwide2xs4_1sample_modified_6x2pt!
cov saved to cov/invcov_romanwide2xs4_1sample_modified_10x2pt!
cov saved to cov/invcov_romanwide2xs4_1sample_modified_8x2pt!


In [22]:
cov_modified.test_eigens()

sorted eigenvalues:  [5.92516112e-44 1.44165906e-43 6.28623801e-43 ... 2.99278104e-11
 4.45824658e-11 6.97935121e-11]
Eigenvalues look good!
